In [3]:
import numpy as np
from pathlib import Path
import re
import pandas as pd
import networkx as nx
from cloudvolume import CloudVolume, Skeleton
from io import StringIO
import os
from brainlit.utils.util import (
    check_type,
    check_size,
)
from brainlit.utils.Neuron_trace import NeuronTrace

/Users/alishakodibagkar/opt/anaconda3/envs/brainlit/lib/python3.8/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


# NeuronTrace Class

The NeuronTrace class takes neuron data either stored in .swc format or located in an s3 bucket, and loads it as a dataframe.

If the neuron is stored in a .swc file, the swc filepath must be provided, and if the neuron is stored in an s3 bucket, the url, segment ID, and mip (resolution to use for scaling) must be provided.

In [2]:
#swc_path = "../../../../../tree_2.swc"
swc_path = "../../../../Manual-GT/8-01_test_1-5/8-01_test_1/tree_2.swc"
s3_path = "s3://open-neurodata/brainlit/brain1_segments"
seg_id = 11
mip = 2

swc_trace = NeuronTrace(swc_path)
s3_trace = NeuronTrace(s3_path,seg_id,mip)

Downloading: 100%|██████████| 1/1 [00:00<00:00, 21.40it/s]


## Methods of NeuronTrace Class

### 1. get_df_arguments

This method returns the arguments of the dataframe - for .swc files these will include offset, color, cc, branch, and for s3 files there should be none


In [3]:
swc_trace.get_df_arguments()

[[73954.8686, 17489.532566, 34340.365689], [1.0, 1.0, 1.0], nan, nan]

In [4]:
s3_trace.get_df_arguments()

[]

### 2. get_df

This method returns the dataframe object for the input neuron

In [5]:
swc_trace.get_df()

,sample,structure,x,y,z,r,parent
0,1,0,-52.589700,-1.448032,-1.228827,1.0,-1
1,2,0,-52.290940,-1.448032,-1.228827,1.0,1
2,3,0,-51.992181,-1.143616,-0.240423,1.0,2
3,4,0,-51.095903,-1.143616,-0.240423,1.0,3
4,5,0,-50.797144,-0.839201,-0.240423,1.0,4
...,...,...,...,...,...,...,...
148,149,0,45.702088,14.381594,-7.159252,1.0,148
149,150,0,46.000847,14.686010,-7.159252,1.0,149
150,151,0,46.897125,14.686010,-7.159252,1.0,150
151,152,0,47.494643,15.294842,-7.159252,1.0,151


In [6]:
s3_trace.get_df()

,sample,structure,x,y,z,r,parent
0,1,0,1104.0,1718.0,1396.0,1.0,-1
1,2,0,1120.0,1703.0,1399.0,1.0,1
2,3,0,1134.0,1691.0,1400.0,1.0,2
3,4,0,1152.0,1675.0,1403.0,1.0,3
4,14,0,1172.0,1655.0,1407.0,1.0,4
...,...,...,...,...,...,...,...
2558,317,0,1308.0,1477.0,1415.0,1.0,354
2559,255,192,1296.0,1476.0,1417.0,1.0,317
2560,194,64,1283.0,1465.0,1420.0,1.0,255
2561,167,0,1275.0,1455.0,1423.0,1.0,194


### 3. get_skel

This method returns the Skeleton object of the dataframe, if the input file is a swc

In [7]:
swc_trace.get_skel(benchmarking=True)

Skeleton(segid=, vertices=(shape=153, float32), edges=(shape=152, uint32), radius=(153, float32), vertex_types=(153, uint8), vertex_color=(153, float32), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])

In [8]:
s3_trace.get_skel()

Downloading: 100%|██████████| 1/1 [00:00<00:00, 16.74it/s]


Skeleton(segid=11, vertices=(shape=2563, float32), edges=(shape=2562, uint32), radius=(2563, float32), vertex_types=(2563, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])

In [9]:
swc_trace.get_skel(benchmarking=True,origin=np.asarray([7,7,7]))

Skeleton(segid=, vertices=(shape=153, float32), edges=(shape=152, uint32), radius=(153, float32), vertex_types=(153, uint8), vertex_color=(153, float32), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])

In [10]:
s3_trace.get_skel(origin=np.asarray([7,7,7]))

Downloading: 100%|██████████| 1/1 [00:00<00:00, 16.07it/s]


Skeleton(segid=11, vertices=(shape=2563, float32), edges=(shape=2562, uint32), radius=(2563, float32), vertex_types=(2563, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])

### 4. get_df_voxel

This method provides the dataframe converted from spatial units to voxel, if spacing and an origin is provided. If origin is not specified, it defaults to (0,0,0)

In [11]:
#swc trace - origin not provided
swc_trace.get_df_voxel(spacing=np.asarray([2,2,2]))

,sample,structure,x,y,z,r,parent
0,1,0,-26,-1,-1,1.0,-1
1,2,0,-26,-1,-1,1.0,1
2,3,0,-26,-1,0,1.0,2
3,4,0,-26,-1,0,1.0,3
4,5,0,-25,0,0,1.0,4
...,...,...,...,...,...,...,...
148,149,0,23,7,-4,1.0,148
149,150,0,23,7,-4,1.0,149
150,151,0,23,7,-4,1.0,150
151,152,0,24,8,-4,1.0,151


In [12]:
#swc trace - origin provided
swc_trace.get_df_voxel(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

,sample,structure,x,y,z,r,parent
0,1,0,-276,-251,-251,1.0,-1
1,2,0,-276,-251,-251,1.0,1
2,3,0,-276,-251,-250,1.0,2
3,4,0,-276,-251,-250,1.0,3
4,5,0,-275,-250,-250,1.0,4
...,...,...,...,...,...,...,...
148,149,0,-227,-243,-254,1.0,148
149,150,0,-227,-243,-254,1.0,149
150,151,0,-227,-243,-254,1.0,150
151,152,0,-226,-242,-254,1.0,151


In [13]:
#s3 trace - origin not provided
s3_trace.get_df_voxel(spacing=np.asarray([2,2,2]))

,sample,structure,x,y,z,r,parent
0,1,0,552,859,698,1.0,-1
1,2,0,560,852,700,1.0,1
2,3,0,567,846,700,1.0,2
3,4,0,576,838,702,1.0,3
4,14,0,586,828,704,1.0,4
...,...,...,...,...,...,...,...
2558,317,0,654,738,708,1.0,354
2559,255,192,648,738,708,1.0,317
2560,194,64,642,732,710,1.0,255
2561,167,0,638,728,712,1.0,194


In [14]:
#s3 trace - origin provided
s3_trace.get_df_voxel(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

,sample,structure,x,y,z,r,parent
0,1,0,302,609,448,1.0,-1
1,2,0,310,602,450,1.0,1
2,3,0,317,596,450,1.0,2
3,4,0,326,588,452,1.0,3
4,14,0,336,578,454,1.0,4
...,...,...,...,...,...,...,...
2558,317,0,404,488,458,1.0,354
2559,255,192,398,488,458,1.0,317
2560,194,64,392,482,460,1.0,255
2561,167,0,388,478,462,1.0,194


### 5. get_graph

This method provides the dataframe in graph format. 
If spacing and origin is specified, the units will be converted from spatial to voxel units prior to making the graph.
If only spacing is specified, origin will be set to (0,0,0)


In [15]:
#swc input
swc_trace.get_graph()

In [16]:
#swc input, only spacing specified, origin defaults to (0,0,0)
swc_trace.get_graph(spacing=np.asarray([2,2,2]))

In [17]:
#swc input, spacing and origin specified
swc_trace.get_graph(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

In [18]:
#s3 input
s3_trace.get_graph()

In [19]:
#s3 input, only spacing specified, origin defaults to (0,0,0)
s3_trace.get_graph(spacing=np.asarray([2,2,2]))

In [20]:
#s3 input, spacing and origin specified
s3_trace.get_graph(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

### 6. get_paths

This method provides the dataframe as a list of paths. If spacing and origin is specified, the units will be converted from spatial to voxel units prior to making the paths. If only spacing is specified, origin will be set to (0,0,0)

In [21]:
#swc input - print 10
swc_trace.get_paths()[0][1:10]

array([[-52, -1, -1],
       [-51, -1, 0],
       [-51, -1, 0],
       [-50, 0, 0],
       [-50, 0, 0],
       [-49, 0, 0],
       [-48, 0, 0],
       [-46, 0, 0],
       [-46, 0, 0]], dtype=object)

In [22]:
#swc input, only spacing specified, origin defaults to (0,0,0)
swc_trace.get_paths(spacing=np.asarray([2,2,2]))[0][1:10]

array([[-26, -1, -1],
       [-26, -1, 0],
       [-26, -1, 0],
       [-25, 0, 0],
       [-25, 0, 0],
       [-25, 0, 0],
       [-24, 0, 0],
       [-23, 0, 0],
       [-23, 0, 0]], dtype=object)

In [23]:
#swc input, spacing and origin specified
swc_trace.get_paths(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))[0][1:10]

array([[-276, -251, -251],
       [-276, -251, -250],
       [-276, -251, -250],
       [-275, -250, -250],
       [-275, -250, -250],
       [-275, -250, -250],
       [-274, -250, -250],
       [-273, -250, -250],
       [-273, -250, -250]], dtype=object)

In [24]:
#s3 input
s3_trace.get_paths()[0][1:10]

array([[1120, 1703, 1399],
       [1134, 1691, 1400],
       [1152, 1675, 1403],
       [1172, 1655, 1407],
       [1184, 1642, 1410],
       [1193, 1629, 1412],
       [1203, 1616, 1415],
       [1213, 1601, 1415],
       [1228, 1581, 1416]])

In [25]:
#s3 input, only spacing specified, origin defaults to (0,0,0)
s3_trace.get_paths(spacing=np.asarray([2,2,2]))[0][1:10]

array([[560, 852, 700],
       [567, 846, 700],
       [576, 838, 702],
       [586, 828, 704],
       [592, 821, 705],
       [596, 814, 706],
       [602, 808, 708],
       [606, 800, 708],
       [614, 790, 708]])

In [26]:
#s3 input, spacing and origin specified
s3_trace.get_paths(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))[0][1:10]

array([[310, 602, 450],
       [317, 596, 450],
       [326, 588, 452],
       [336, 578, 454],
       [342, 571, 455],
       [346, 564, 456],
       [352, 558, 458],
       [356, 550, 458],
       [364, 540, 458]])

### 7. generate_df_subset

This method reads a new subset dataframe, taking in a list of voxels. An option was added to provide a subset of the neuron rather than the entire neuron (by providing subneuron_start and subneuron_end)

In [27]:
#swc input, no subneuron_start and subneuron_end

#generate vox_in_img_list
my_list = []
for i in range(len(swc_trace.get_df())):
    my_list.append(10)
vox_in_img_list = [my_list,my_list,my_list]

swc_trace.generate_df_subset(vox_in_img_list)

,sample,structure,x,y,z,r,parent
0,1,0,10,10,10,1.0,-1
1,2,0,10,10,10,1.0,1
2,3,0,10,10,10,1.0,2
3,4,0,10,10,10,1.0,3
4,5,0,10,10,10,1.0,4
...,...,...,...,...,...,...,...
148,149,0,10,10,10,1.0,148
149,150,0,10,10,10,1.0,149
150,151,0,10,10,10,1.0,150
151,152,0,10,10,10,1.0,151


In [28]:
#swc input, subneuron_start and subneuron_end specified

subneuron_start = 5
subneuron_end = 8

#generate vox_in_img_list
my_list = []
for i in range(subneuron_end-subneuron_start):
    my_list.append(10)
vox_in_img_list_2 = list([my_list,my_list,my_list])

swc_trace.generate_df_subset(vox_in_img_list_2,subneuron_start,subneuron_end)

,sample,structure,x,y,z,r,parent
5,6,0,10,10,10,1.0,5
6,7,0,10,10,10,1.0,6
7,8,0,10,10,10,1.0,7


In [29]:
#s3 input, no subneuron_start and subneuron_end

#generate vox_in_img_list
my_list = []
for i in range(len(s3_trace.get_df())):
    my_list.append(10)
vox_in_img_list_3 = [my_list,my_list,my_list]

s3_trace.generate_df_subset(vox_in_img_list_3)

,sample,structure,x,y,z,r,parent
0,1,0,10,10,10,1.0,-1
1,2,0,10,10,10,1.0,1
2,3,0,10,10,10,1.0,2
3,4,0,10,10,10,1.0,3
4,14,0,10,10,10,1.0,4
...,...,...,...,...,...,...,...
2558,317,0,10,10,10,1.0,354
2559,255,192,10,10,10,1.0,317
2560,194,64,10,10,10,1.0,255
2561,167,0,10,10,10,1.0,194


In [30]:
#s3 input, subneuron_start and subneuron_end specified

subneuron_start = 5
subneuron_end = 8

#generate vox_in_img_list
my_list = []
for i in range(subneuron_end-subneuron_start):
    my_list.append(10)
vox_in_img_list_4 = [my_list,my_list,my_list]

s3_trace.generate_df_subset(vox_in_img_list_4,subneuron_start,subneuron_end)

,sample,structure,x,y,z,r,parent
5,29,0,10,10,10,1.0,14
6,44,0,10,10,10,1.0,29
7,57,0,10,10,10,1.0,44


### 8. get_bfs_subgraph

This method creates a spanning subgraph from a seed node and parent graph created from the dataframes using BFS. The seed node ID and max depth for BFS should be specified. A dataframe storing indices can be specified. If spacing and origin is specified, the units will be converted from spatial to voxel units. If only spacing is specified, origin will be set to (0,0,0).


In [31]:
#swc input, specify node_id and depth
swc_trace.get_bfs_subgraph(node_id=11,depth=2)

(<networkx.classes.digraph.DiGraph at 0x7f81a8390190>,
 <networkx.classes.digraph.DiGraph at 0x7f81a8413fa0>)

In [32]:
#swc input, provide a dataframe
swc_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df())

(<networkx.classes.digraph.DiGraph at 0x7f81a8390070>,
 <networkx.classes.digraph.DiGraph at 0x7f819ff4d3a0>)

In [33]:
#swc input, add spacing 
swc_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df(),spacing=np.asarray([2,2,2]))

(<networkx.classes.digraph.DiGraph at 0x7f81a8390b80>,
 <networkx.classes.digraph.DiGraph at 0x7f81a832c040>)

In [34]:
#swc input, add spacing and origin
swc_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df(),spacing=np.asarray([2,2,2]),origin=np.asarray([50,50,50]))

(<networkx.classes.digraph.DiGraph at 0x7f81a837b7c0>,
 <networkx.classes.digraph.DiGraph at 0x7f81a8390940>)

In [35]:
#s3 input, specify node_id and depth
s3_trace.get_bfs_subgraph(node_id=11,depth=2)

(<networkx.classes.digraph.DiGraph at 0x7f81a8393460>,
 <networkx.classes.digraph.DiGraph at 0x7f81a8393400>)

In [36]:
#s3 input, provide a dataframe
s3_trace.get_bfs_subgraph(node_id=11,depth=2,df=swc_trace.get_df())

(<networkx.classes.digraph.DiGraph at 0x7f81a8393310>,
 <networkx.classes.digraph.DiGraph at 0x7f81a8393160>)

In [37]:
#s3 input, add spacing 
s3_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df(),spacing=np.asarray([2,2,2]))

(<networkx.classes.digraph.DiGraph at 0x7f81a837be80>,
 <networkx.classes.digraph.DiGraph at 0x7f81a8393a90>)

In [38]:
#s3 input, add spacing and origin
s3_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df(),spacing=np.asarray([2,2,2]),origin=np.asarray([50,50,50]))

(<networkx.classes.digraph.DiGraph at 0x7f818a302ee0>,
 <networkx.classes.digraph.DiGraph at 0x7f81a837b7f0>)

### 9. get_sub_neuron

This method returns a sub-neuron in graph format with node coordinates bounded by a bounding box with start and end. If spacing and origin is specified, the units will be converted from spatial to voxel units. If only spacing is specified, origin will be set to (0,0,0).

In [39]:
bounding_box=[[1,2,4],[1,2,3]]

#swc input, no spacing and origin
swc_trace.get_sub_neuron(bounding_box)

In [40]:
#swc input, spacing specified
swc_trace.get_sub_neuron(bounding_box,spacing=np.asarray([2,2,2]))

In [41]:
#swc input, spacing and origin specified
swc_trace.get_sub_neuron(bounding_box,spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

In [42]:
#s3 input, no spacing and origin
s3_trace.get_sub_neuron(bounding_box)

In [43]:
#s3 input, spacing specified
s3_trace.get_sub_neuron(bounding_box,spacing=np.asarray([2,2,2]))

In [44]:
#s3 input, spacing and origin specified
s3_trace.get_sub_neuron(bounding_box,spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

### 10. get_sub_neuron_paths

This method returns a sub-neuron in paths format with node coordinates bounded by a bounding box with start and end. If spacing and origin is specified, the units will be converted from spatial to voxel units. If only spacing is specified, origin will be set to (0,0,0). 

In [7]:
bounding_box=[[1,2,4],[1,2,3]]

#swc input, no spacing and origin
swc_trace.get_sub_neuron_paths(bounding_box)

array([], dtype=object)

In [8]:
#swc input, spacing specified
swc_trace.get_sub_neuron_paths(bounding_box,spacing=np.asarray([2,2,2]))

array([], dtype=object)

In [9]:
#swc input, spacing and origin specified
swc_trace.get_sub_neuron_paths(bounding_box,spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

array([], dtype=object)

In [12]:
#s3 input, no spacing and origin
s3_trace.get_sub_neuron_paths(bounding_box)

array([], dtype=object)

In [13]:
#s3 input, spacing specified
s3_trace.get_sub_neuron_paths(bounding_box,spacing=np.asarray([2,2,2]))

array([], dtype=object)

In [14]:
#s3 input, spacing and origin specified
s3_trace.get_sub_neuron_paths(bounding_box,spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

array([], dtype=object)

### 11. ssd

Computes significant spatial distance metric between two traces (for APP1)

In [45]:
pts1 = swc_trace.get_paths()[0][1:10]
pts2 = swc_trace.get_paths()[0][11:20]

NeuronTrace.ssd(pts1,pts2)

6.247937554557103